# In this notebook an XGBoost Regression Model is trained on the metadata (grabbed from Spotify) of various songs from the music_info.csv
## The goal is to create a model that predicts whether the given song is similar to Mr. Brightside by The Killers
## This could be repeated for any other song as long as we have the necessary data

In [96]:
#The csv file is imported as a data frame through Pandas

In [1]:
import pandas as pd

In [2]:
Musicinfo_df = pd.read_csv("music_info.csv")
Musicinfo_df.head(5)

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,09ZQ5TmUG8TSL56n0knqrj,"rock, alternative, indie, alternative_rock, in...",NaN,2004,222200,0.355,...,1,-4.360,1,0.0746,0.001190,0.000000,0.0971,0.240,148.114,4
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",NaN,2006,258613,0.409,...,2,-4.373,1,0.0336,0.000807,0.000000,0.2070,0.651,174.426,4
2,TROUVHL128F426C441,Come as You Are,Nirvana,https://p.scdn.co/mp3-preview/a1c11bb1cb231031...,0keNu0t0tqsWtExGM3nT1D,"rock, alternative, alternative_rock, 90s, grunge",RnB,1991,218920,0.508,...,4,-5.783,0,0.0400,0.000175,0.000459,0.0878,0.543,120.012,4
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,https://p.scdn.co/mp3-preview/399c401370438be4...,0ancVQ9wEcHVd0RrGICTE4,"rock, alternative, indie, alternative_rock, in...",NaN,2004,237026,0.279,...,9,-8.851,1,0.0371,0.000389,0.000655,0.1330,0.490,104.560,4
4,TRLNZBD128F935E4D8,Creep,Radiohead,https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...,01QoK9DA7VTeTSE3MNzp4I,"rock, alternative, indie, alternative_rock, in...",RnB,2008,238640,0.515,...,7,-9.935,1,0.0369,0.010200,0.000141,0.1290,0.104,91.841,4


In [ ]:
#certain columns are dropped from the data frame because they don't really have much to do with checking for similarity

In [3]:
Reduced_music_info_df = Musicinfo_df.drop(columns = ['track_id','spotify_preview_url','spotify_id','tags','genre','year','duration_ms'])
Reduced_music_info_df.head(5)

,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,Mr. Brightside,The Killers,0.355,0.918,1,-4.360,1,0.0746,0.001190,0.000000,0.0971,0.240,148.114,4
1,Wonderwall,Oasis,0.409,0.892,2,-4.373,1,0.0336,0.000807,0.000000,0.2070,0.651,174.426,4
2,Come as You Are,Nirvana,0.508,0.826,4,-5.783,0,0.0400,0.000175,0.000459,0.0878,0.543,120.012,4
3,Take Me Out,Franz Ferdinand,0.279,0.664,9,-8.851,1,0.0371,0.000389,0.000655,0.1330,0.490,104.560,4
4,Creep,Radiohead,0.515,0.430,7,-9.935,1,0.0369,0.010200,0.000141,0.1290,0.104,91.841,4


In [4]:
#Spotify API Track Attributes
#acousticness - A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.
#danceability - Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.
#energy - Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.
#instrumentalness - Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.
#key - The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1.
#liveness - Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live.
#loudness - The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typically range between -60 and 0 db.
#mode - Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0.
#speechiness - Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.
#tempo - The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.
#time_signature - An estimated time signature. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). The time signature ranges from 3 to 7 indicating time signatures of "3/4", to "7/4".
#valence - A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).

In [5]:
#attributes we choose to use in similarity algorithms: 
#danceability (score), energy (score), key (classes), mode (classes), tempo (score), valence (score)
#these features were selected because they all take into account fundamental musical properties that characterize the way the song "sounds" 

In [6]:
Final_music_collection_df = Reduced_music_info_df.drop(columns = ['loudness','speechiness','acousticness','instrumentalness','liveness','time_signature'])
Final_music_collection_df.head(5)

,name,artist,danceability,energy,key,mode,valence,tempo
0,Mr. Brightside,The Killers,0.355,0.918,1,1,0.240,148.114
1,Wonderwall,Oasis,0.409,0.892,2,1,0.651,174.426
2,Come as You Are,Nirvana,0.508,0.826,4,0,0.543,120.012
3,Take Me Out,Franz Ferdinand,0.279,0.664,9,1,0.490,104.560
4,Creep,Radiohead,0.515,0.430,7,1,0.104,91.841


In [97]:
Final_music_collection_df.to_csv('music_info_chosenattributes.csv', index=True)

In [ ]:
#Create the similarity label to Track[0] "Mr. Brightside" by The Killers

In [35]:
#the closer to 0, the more dissimilar; the closer to 1, the more similar
#The below labels pretty much just take a difference of the danceability, energy, and valence scores while for key and mode, we assign a value of 1 if two songs are the same key or mode and 0 if not
def danceability_similarity(danceability_score1, danceability_score2):
    dance_similarity_score = 1 - abs(danceability_score1 - danceability_score2)
    return dance_similarity_score

def energy_similarity(energy_score1, energy_score2):
    energy_similarity_score = 1 - abs(energy_score1 - energy_score2)
    return energy_similarity_score

def key_similarity(key_score1, key_score2):
    if key_score1 == key_score2:
        key_similarity_score = 1
    else:
        key_similarity_score = 0
    return key_similarity_score

def mode_similarity(mode_score1, mode_score2):
    if mode_score1 == mode_score2:
        mode_similarity_score = 1
    else:
        mode_similarity_score = 0
    return mode_similarity_score

def valence_similarity(valence_score1, valence_score2):
    valence_similarity_score = 1 - abs(valence_score1 - valence_score2)
    return valence_similarity_score

In [ ]:
#We have to scale the tempo feature to be in between 0 and 1
#The min max scaler was chosen as the scaler

In [34]:
from sklearn import preprocessing
x = Final_music_collection_df['tempo'].values.reshape(-1, 1)
min_max_scaler = preprocessing.MinMaxScaler()
tempo_list_transformed = min_max_scaler.fit_transform(x)
Final_music_collection_df['tempo'] = pd.Series(tempo_list_transformed[:, 0]).tolist()
Final_music_collection_df.head(5)

,name,artist,danceability,energy,key,mode,valence,tempo
0,Mr. Brightside,The Killers,0.355,0.918,1,1,0.240,0.619996
1,Wonderwall,Oasis,0.409,0.892,2,1,0.651,0.730137
2,Come as You Are,Nirvana,0.508,0.826,4,0,0.543,0.502363
3,Take Me Out,Franz Ferdinand,0.279,0.664,9,1,0.490,0.437682
4,Creep,Radiohead,0.515,0.430,7,1,0.104,0.384441


In [ ]:
#the similarity score for tempo after it has been scaled is now just the difference of the tempos between two songs
#the overall similarity score is determine by taking a average of the other 6 mentioned similarity scores

In [54]:
def tempo_similarity(tempo_score1, tempo_score2):
    tempo_similarity_score = 1- abs(tempo_score1 - tempo_score2)
    return tempo_similarity_score

def average_similarity(dance_similarity_score,energy_similarity_score,key_similarity_score,mode_similarity_score,tempo_similarity_score,valence_similarity_score):
    average_similarity_score = (dance_similarity_score + energy_similarity_score + key_similarity_score + mode_similarity_score + tempo_similarity_score + valence_similarity_score)/6
    return average_similarity_score

In [ ]:
#the columns of the previous features are put into lists

In [55]:
danceability_list = Final_music_collection_df.danceability.tolist()
energy_list = Final_music_collection_df.energy.tolist()
key_list = Final_music_collection_df.key.tolist()
mode_list = list(Final_music_collection_df['mode'])
valence_list = Final_music_collection_df.valence.tolist()
tempo_list = Final_music_collection_df.tempo.tolist()

In [ ]:
#We generate a similarity score for all songs in the data frame when compared to Mr.Brightside but we could repeat this process for any other song with the same information

In [56]:
similarity_list = []
for i in range(0, len(Final_music_collection_df)):
    similarity_list.append(average_similarity(danceability_similarity(danceability_list[0], danceability_list[i]),energy_similarity(energy_list[0], energy_list[i]),key_similarity(key_list[0], key_list[i]),mode_similarity(mode_list[0], mode_list[i]),tempo_similarity(tempo_list[0], tempo_list[i]),valence_similarity(valence_list[0], valence_list[i])))
Final_music_collection_df['similarity_to_MrBrightside'] = similarity_list
Final_music_collection_df.head(5)

,name,artist,danceability,energy,key,mode,valence,tempo,similarity_to_MrBrightside
0,Mr. Brightside,The Killers,0.355,0.918,1,1,0.240,0.619996,1.000000
1,Wonderwall,Oasis,0.409,0.892,2,1,0.651,0.730137,0.733143
2,Come as You Are,Nirvana,0.508,0.826,4,0,0.543,0.502363,0.555728
3,Take Me Out,Franz Ferdinand,0.279,0.664,9,1,0.490,0.437682,0.706281
4,Creep,Radiohead,0.515,0.430,7,1,0.104,0.384441,0.663407


In [ ]:
#A vector of the desired features is generated for each songs called "X_vector"

In [78]:
Final_music_collection_df['X_vector'] = [[Final_music_collection_df.danceability.iloc[i],Final_music_collection_df.energy.iloc[i],Final_music_collection_df.key.iloc[i],Final_music_collection_df['mode'].iloc[i],Final_music_collection_df.valence.iloc[i],Final_music_collection_df.tempo.iloc[i]] for i in range(0, len(Final_music_collection_df))]
Final_music_collection_df.head(5)

,name,artist,danceability,energy,key,mode,valence,tempo,similarity_to_MrBrightside,X_vector
0,Mr. Brightside,The Killers,0.355,0.918,1,1,0.240,0.619996,1.000000,"[0.355, 0.918, 1, 1, 0.24, 0.6199962326545135]"
1,Wonderwall,Oasis,0.409,0.892,2,1,0.651,0.730137,0.733143,"[0.409, 0.892, 2, 1, 0.651, 0.7301366709223718]"
2,Come as You Are,Nirvana,0.508,0.826,4,0,0.543,0.502363,0.555728,"[0.508, 0.826, 4, 0, 0.543, 0.5023629628079282]"
3,Take Me Out,Franz Ferdinand,0.279,0.664,9,1,0.490,0.437682,0.706281,"[0.279, 0.664, 9, 1, 0.49, 0.4376818267439671]"
4,Creep,Radiohead,0.515,0.430,7,1,0.104,0.384441,0.663407,"[0.515, 0.43, 7, 1, 0.104, 0.3844408631407104]"


In [ ]:
#we split the data into a train and test set

In [79]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(Final_music_collection_df, test_size=0.2,random_state=123)

In [80]:
train

,name,artist,danceability,energy,key,mode,valence,tempo,similarity_to_MrBrightside,X_vector
43104,The Pig Keeper's Daughter,PsyOpus,0.314,0.983,11,0,0.335,0.469771,0.608129,"[0.314, 0.983, 11, 0, 0.335, 0.46977123841018026]"
40948,Olympic,Millencolin,0.293,0.927,11,0,0.583,0.399050,0.560842,"[0.293, 0.927, 11, 0, 0.583, 0.39904979174951344]"
30803,Change the System,Rebelution,0.647,0.720,0,0,0.666,0.330593,0.465766,"[0.647, 0.72, 0, 0, 0.666, 0.33059293832018255]"
24226,Blink of an Eye,Damageplan,0.484,0.851,5,0,0.450,0.469876,0.573980,"[0.484, 0.851, 5, 0, 0.45, 0.4698758868959167]"
5079,The Enemy,Paradise Lost,0.236,0.911,2,0,0.280,0.530903,0.624151,"[0.236, 0.911, 2, 0, 0.28, 0.5309026978379623]"
...,...,...,...,...,...,...,...,...,...,...
7763,Mechanical Animals,Marilyn Manson,0.432,0.913,4,0,0.212,0.564600,0.639101,"[0.432, 0.913, 4, 0, 0.212, 0.5645995102450868]"
15377,Chemistry,UNKLE,0.217,0.711,7,1,0.187,0.315143,0.716191,"[0.217, 0.711, 7, 1, 0.187, 0.31514263588605873]"
17730,Steeler,Judas Priest,0.495,0.875,9,1,0.468,0.440047,0.734842,"[0.495, 0.875, 9, 1, 0.468, 0.44004688252160995]"
28030,Champion Nibble,Mr. Scruff,0.766,0.788,10,0,0.676,0.447962,0.475161,"[0.766, 0.788, 10, 0, 0.676, 0.44796249398271215]"


In [ ]:
#create the list of X_vector and y target values for the training set

In [83]:
X_train = train.X_vector.tolist()
y_train = train.similarity_to_MrBrightside.tolist()

In [ ]:
#Import the XGBoot Regressor model and fit it to the training set

In [87]:
from xgboost import XGBRegressor
model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=123)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=123, ...)

In [81]:
test

,name,artist,danceability,energy,key,mode,valence,tempo,similarity_to_MrBrightside,X_vector
37518,Mein Herz,Letzte Instanz,0.548,0.815,5,1,0.612,0.418950,0.688492,"[0.548, 0.815, 5, 1, 0.612, 0.4189497477971494]"
27861,Woman Friend,Chromeo,0.887,0.598,8,1,0.816,0.481379,0.572230,"[0.887, 0.598, 8, 1, 0.816, 0.481378848448063]"
10542,A Minha Menina,Os Mutantes,0.473,0.823,7,1,0.664,0.385571,0.688096,"[0.473, 0.823, 7, 1, 0.664, 0.3855710667866637]"
49943,Feuerbringer,Wolfchant,0.384,0.940,10,1,0.503,0.611637,0.779607,"[0.384, 0.94, 10, 1, 0.503, 0.611636911613889]"
12581,Burden of Tomorrow,The Tallest Man on Earth,0.488,0.464,6,1,0.486,0.654032,0.688827,"[0.488, 0.464, 6, 1, 0.486, 0.654032106155424]"
...,...,...,...,...,...,...,...,...,...,...
32746,The Singer Not The Song,The Rolling Stones,0.718,0.581,0,1,0.884,0.463689,0.583282,"[0.718, 0.581, 0, 1, 0.884, 0.46368906841918]"
10779,Factory Girl,The Rolling Stones,0.533,0.434,2,1,0.464,0.467800,0.660301,"[0.533, 0.434, 2, 1, 0.464, 0.46779966093890624]"
34718,Sarah Smiles,Panic! at the Disco,0.332,0.837,5,1,0.551,0.717223,0.747962,"[0.332, 0.837, 5, 1, 0.551, 0.7172230477824987]"
19551,A Moment Forever,Volbeat,0.544,0.966,2,1,0.397,0.602160,0.764694,"[0.544, 0.966, 2, 1, 0.397, 0.6021599447455996]"


In [ ]:
#generate the X_vector and y targeting variables for the test set and use the model to predict on the X_vector test set

In [89]:
X_test = test.X_vector.tolist()

In [90]:
predictions = model.predict(X_test)

In [91]:
y_test = test.similarity_to_MrBrightside.tolist()

In [ ]:
#Its a pretty low RMSE we get, the model makes more accurate predictions and fits the data well

In [94]:
#evaluating our model 
from sklearn.metrics import mean_squared_error as MSE 
rmse = np.sqrt(MSE(y_test, predictions)) 
print("RMSE : % f" %(rmse))

RMSE :  0.004909
